In [ ]:
!pip install -U shortid rdflib PyYAML

In [1]:
import yaml
import glob
import pandas as pd
from lib.utils import translate
from rdflib import plugin, ConjunctiveGraph, URIRef
from rdflib.store import Store

In [2]:
vocab = dict()
vocab[None] = URIRef('http://envri.eu/ns/NULL')
vocab['NULL'] = URIRef('http://envri.eu/ns/NULL')

store = plugin.get('IOMemory', Store)()

g = ConjunctiveGraph(store)
g.bind('envri', 'http://envri.eu/ns/')
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.bind('skos', 'http://www.w3.org/2004/02/skos/core#')

with open('vocab.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        vocab[key] = URIRef(value)

with open('fairmapping.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        g.add((vocab[key], vocab['relatesTo'], URIRef(value)))
    
for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f: 
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            translate(store, document, vocab)
    
g.serialize(destination='data.trig', format='trig')

KeyError: 'G'